## A quick introduction to HDF5 files for seismic

We will read a preformed dataset from a NumPy file, then save the dataset as an HDF5 file.

This notebook follows on from `Intro_to_seismic.ipynb`.

## What are HDF5 files?

'HDF' stands for _hierarchical data format_. 

An HDF5 **File** can contain multiple **Group** and **Dataset** items. 

- A **Group** is a bit like a file system directory, and a bit like a Python dictionary. Groups can be arbitrarily nested (hence the _H_ in HDF). They can contain Datasets, or other Groups. The **File** is, for all intents and purposes, a root-level group.

- A **Dataset** is a lot like a NumPy array. It's an n-dimensional (hyper-)rectangular data object, containing elements of homogenous type.

Both Groups and Datasets can have **Attributes**: a dictionary-like `attrs` object attached to them, which holds metadata.

HDF5 files support compression, error detection, metadata, and other useful things. They also support chunking, which can dramatically speed up data access on large files ([more about this](http://geology.beer/2015/02/10/hdf-for-large-arrays/)). 

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
volume = np.load('../data/F3_volume_3x3_16bit.npy')

In [ ]:
volume.shape

## Saving to HDF5

We'll be using the HDF5 library, via [the `h5py` Python library](http://docs.h5py.org/en/stable/index.html).

If you need to, you can install `h5py` from the Notebook with: `!conda install h5py`. 

In [ ]:
import h5py

The `h5py.File` object is a context manager, so we will use it that way. Let's write our seismic volume to an HDF5 file:

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'w') as f:
    dset = f.create_dataset("volume", data=volume)

That has saved a file:

In [ ]:
%ls -l ../data/f3_seismic.hdf5

This is a bit bigger than the array:

In [ ]:
volume.nbytes

So there's a little bit more overhead than a NumPy binary file:

In [ ]:
%ls -l ../data/F3_volume_3x3_16bit.npy

We can compress the dataset (there's not a lot you can do on this data, but it does squeeze down a bit) &mdash; this reduces the file size, but takes a little time.

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'w') as f:
    dset = f.create_dataset("volume", data=volume, compression='gzip')

In [ ]:
%ls -l ../data/f3_seismic.hdf5

Let's add a bit of metadata:

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'w') as f:
    grp = f.create_group('amplitude_group')

    dset = f.create_dataset("amplitude_group/timeseries_dataset", data=volume)
    
    # Let's add some attributes (metadata) to the root-level File object...
    f.attrs['survey'] = 'F3'
    f.attrs['location'] = 'Netherlands'
    f.attrs['owner'] = 'NAM/NLOG/TNO/dGB'
    f.attrs['licence'] = 'CC-BY-SA'
    
    # ...and to the group...
    grp.attrs['kind'] = 'Raw seismic amplitude, no units'
    
    # ...and to the dataset.
    dset.attrs['domain'] = 'time'
    dset.attrs['dt'] = '0.004'

## Reading HDF5

You will need to poke around a bt to figure out what the paths and datasets are. At first, it's best just to read the metadata, unless you know what you're expecting to find.

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print([k for k in f.keys()])
    print([k for k in f.attrs])

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print([k for k in f['amplitude_group'].keys()])
    print([k for k in f['amplitude_group'].attrs])

Once you've figured out what you want, you can read the data:

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    data = f['amplitude_group/timeseries_dataset'][:]

In [ ]:
data.shape

But we can also read slices from the volume without loading it all into memory:

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    data = f['amplitude_group/timeseries_dataset'][..., 200]

In [ ]:
plt.imshow(data)

## Adding a new dataset to an HDF5 file

Let's add a frequency cube with dimensions inlines, crosslines, frequency.

In [ ]:
import scipy.signal

freqs, Pxx_den = scipy.signal.welch(volume, fs=250)

In [ ]:
plt.imshow(Pxx_den[:, :, 20])

In [ ]:
Pxx_den.shape

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r+') as f:

    dset = f.create_dataset("amplitude_group/frequency_dataset", data=Pxx_den)
    
    # This time we only want to add metadata to the dataset.
    dset.attrs['domain'] = 'frequency'
    dset.attrs['df'] = str(125 / 128)

The HDF5 file has grown commensurately:

In [ ]:
%ls -l ../data/f3_seismic.hdf5

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print([k for k in f['amplitude_group'].keys()])
    print([k for k in f['amplitude_group/frequency_dataset'].attrs])

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print('domain =', f['amplitude_group/frequency_dataset'].attrs['domain'])
    print('df =', f['amplitude_group/frequency_dataset'].attrs['df'])

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print(np.mean(f['amplitude_group/frequency_dataset']))

## Modifying an existing dataset

You can't point a name at a new dataset, but you can change the values inside a dataset &mdash; as long as it doen't change shape.

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r+') as f:

    data = f["amplitude_group/frequency_dataset"]
    data[...] = np.sqrt(Pxx_den)

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print(np.mean(f['amplitude_group/frequency_dataset']))

If it changes shape, you'll have to delete it and add it again:

In [ ]:
data = np.mean(Pxx_den, axis=1)
data.shape

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r+') as f:

    del(f["amplitude_group/frequency_dataset"])

    dset = f.create_dataset("amplitude_group/frequency_dataset", data=data)
    
    # This time we only want to add metadata to the dataset.
    dset.attrs['domain'] = 'frequency'
    dset.attrs['df'] = str(125 / 128)

In [ ]:
with h5py.File('../data/f3_seismic.hdf5', 'r') as f:
    print(f['amplitude_group/frequency_dataset'].shape)

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2018</p>
</div>